In [1]:
import pickle, pandas as pd, numpy as np, re

In [2]:
entdicf = "entdic3hops.pkl"
reldicf = "reldic3hops.pkl"
triplesf = "triples3hops.pd.pkl"
allrelsf = "allrel3hops.pd.pkl"
typesf = "../dbpedia/dbpcore.nt.types.dok"
bigentdicf = "../dbpedia/dbpcore.nt.entdic.ssd"
rhsdicf = "rhsdic.pkl"
inclents = False

In [3]:
entdic = pickle.load(open(entdicf))
reldic = pickle.load(open(reldicf))
triples = pickle.load(open(triplesf))
allrels = pickle.load(open(allrelsf))
reventdic = {v: k for k, v in entdic.items()}
# load dictionaries
def loadDic(entdicf):
    entdic = {}
    for line in open(entdicf):
        splits = line.split(" ")
        entdic[splits[0]] = int(splits[1])
    return entdic

# load big dictionaries
bigentdic = loadDic(bigentdicf)
print "big entity dict loaded"
'''bigreldic = loadDic(bigreldicf)
print "big relation dict loaded"'''

big entity dict loaded


'bigreldic = loadDic(bigreldicf)\nprint "big relation dict loaded"'

In [4]:
reventdic = dict([(v, k) for k, v in entdic.items()])
revreldic = dict({(v, k) for k, v in reldic.items()})

#### save triple tensor

In [75]:
triplesdf = triples.copy()
triplesdf.columns = ["x","z","y"]


In [76]:
# save fulltensor as .ssd
name = "tripletensor"
name += ".ssd"
with open(name, "w") as outf:
    for row in triplesdf.iterrows():
        row = row[1]
        outf.write("%d %d %d %f\n" % (row.z, row.x, row.y, 1.0))

In [5]:
def getreconstr(reventdic, revreldic):
    def reconstr(row):
        return reventdic[row.s], revreldic[row.r], reventdic[row.o]
    return reconstr
def checkslice(reventdic, revreldic):
    def inner(tslice):
        recon = getreconstr(reventdic, revreldic)
        for row in tslice.iterrows():
            print recon(row[1])
    return inner

In [6]:
tslice = triples[0:15]
#checkslice(reventdic, revreldic)(tslice)

## TYPES

In [7]:
# TYPES
allents = entdic.keys()
allentbidxs = set(map(lambda x: bigentdic[x], allents))
# go through the types file
enttypes = {}
c = 0
typesused = set()
for line in open(typesf):
    c += 1
    if c % 1000000 == 0:
        print c
    ns = [int(x) for x in line.split(" ")]
    if len(ns) == 2 and ns[0] in allentbidxs:
        if ns[0] not in enttypes:
            enttypes[ns[0]] = set()
        enttypes[ns[0]].add(ns[1])
        typesused.add(ns[1])
revbigentdic = {v: k for k, v in bigentdic.items()}

1000000
2000000
3000000
4000000
5000000
6000000
7000000


In [8]:
ntypesused = map(lambda x: revbigentdic[x], typesused)
ntypesused = sorted(ntypesused)
#ntypesidx = range(max(newreldic.values()) + 1, max(newreldic.values()) + 1 + len(ntypesused))
ntypesidx = range(len(ntypesused))
typdic = dict(zip(ntypesused, ntypesidx))
revtypdic = dict([(v, k) for k, v in typdic.items()])

In [9]:
def maketypelist(enttypes):
    for et in enttypes.items():
        for v in et[1]:
            yield (et[0], v)

nenttypes = [i for i in maketypelist(enttypes)]
enttypesdf = pd.DataFrame(nenttypes)
enttypesdf.columns = ["e", "t"]
enttypesdf["e"] = enttypesdf["e"].map(lambda x: entdic[revbigentdic[x]])
enttypesdf["t"] = enttypesdf["t"].map(lambda x: typdic[revbigentdic[x]])

In [10]:
fet = tuple(enttypesdf.iloc[500])
print revtypdic.keys()
reventdic[fet[0]], revtypdic[fet[1]]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129]


('http://dbpedia.org/resource/Verona',
 'http://dbpedia.org/ontology/Settlement')

## BUILDING TENSOR

In [11]:
# statistics of used variables
def dicstats(x):
    return min(x), max(x)
print map(dicstats, [entdic.values()])
print map(dicstats, [reldic.values()])
print map(dicstats, [typdic.values()])

print triples.describe()
print enttypesdf.describe()
print allrels.describe()

[(0, 11297)]
[(1, 500)]
[(0, 129)]
                  s             r             o
count  34582.000000  34582.000000  34582.000000
mean    5592.967324     17.638309   5100.189549
std     3268.364253     27.972726   3544.182080
min        1.000000      4.000000      0.000000
25%     2757.000000      4.000000   1832.000000
50%     5478.000000      4.000000   4398.000000
75%     8455.000000     26.000000   8323.000000
max    11297.000000    409.000000  11297.000000
                  e             t
count  24123.000000  24123.000000
mean    5661.119471     84.923724
std     3293.721605     38.582510
min        0.000000      0.000000
25%     2776.000000     58.000000
50%     5660.000000     99.000000
75%     8549.000000    129.000000
max    11295.000000    129.000000
                  e              r
count  484893.00000  484893.000000
mean     5470.54008     115.903591
std      3240.65118     126.624463
min         0.00000       1.000000
25%      2661.00000      18.000000
50%      5397.000

### make new reldic that assign reverse relations in the same (extended) index space and reindex allrels.df

In [12]:
# shift inverse
maxid = max(reldic.values())+1
print maxid
newreldic = [(k, v) for k, v in reldic.items()]   # copy
newreldic += [("-"+k, v+maxid) for k, v in reldic.items()] # extend with inverse
newreldic = dict(newreldic)

501


In [13]:
max(newreldic.values()), min(newreldic.values())

(1001, 1)

In [14]:
# reindex inverse
allrels
allrelsnewinv = allrels[allrels.d == "R"]
allrelsnewinv.r += maxid
allrelsnew = pd.concat([allrels[allrels.d == "F"][["e","r"]], allrelsnewinv[["e","r"]]])

/home/denis/dev/semdev/pandas/pandas/core/generic.py:2306: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [15]:
#allrelsnew

In [16]:
#allrelsnew

In [17]:
# shift type dic after entities and rels and reindex
maxid = max(newreldic.values()) + 1
idxtypstart = maxid
newtypdicshift = {k: v + maxid for k, v in typdic.items()}
newenttypshift = enttypesdf.copy()
newenttypshift.t += maxid

In [18]:
revtypdicshift = {v: k for k, v in newtypdicshift.items()}

tslice = newenttypshift[0:500]
for _, row in tslice.iterrows():
    pass
    #print reventdic[row.e], revtypdicshift[row.t]

In [19]:
ydic = newreldic.copy()
ydic.update(newtypdicshift)

In [20]:
len(ydic), max(ydic.values())

(1130, 1131)

In [21]:
revydic = {v: k for k, v in ydic.items()}

In [22]:
xdic = entdic.copy()
revxdic = {v: k for k, v in xdic.items()}

In [23]:
zdic = newreldic.copy()
revzdic = {v: k for k, v in zdic.items()}

In [24]:
zdic["http://dbpedia.org/ontology/spouse"]

27

to save:

In [25]:
print len(ydic) # for y
print len(xdic)  # for x
print len(zdic)

1130
11298
1000


## Actually building tensor

In [26]:
# sources: triples, allrels, types
newenttypshift.columns = ["e", "r"]
allrelsandtypes = pd.concat([allrelsnew, newenttypshift])

In [27]:
# take triples, replace "o" with all relations (from allrels) that "o" has
transfwd = triples.merge(allrelsandtypes, left_on="o", right_on="e")
transfwd.drop(["e"], axis=1, inplace=True)

In [71]:
transfwd

,s,r_x,o,r_y
0,417,31,9145,1
1,417,31,9145,2
2,417,31,9145,4
3,417,31,9145,41
4,417,31,9145,23
5,417,31,9145,107
6,417,31,9145,27
7,417,31,9145,186
8,417,31,9145,408
9,417,31,9145,12


In [29]:
tslice = transfwd[:20]
for i, row in tslice.iterrows():
    pass
    print revxdic[row.s], revzdic[row.r_x], revxdic[row.o], revydic[row.r_y]

http://dbpedia.org/resource/Citizen_Kane http://dbpedia.org/ontology/director http://dbpedia.org/resource/Orson_Welles http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://dbpedia.org/resource/Citizen_Kane http://dbpedia.org/ontology/director http://dbpedia.org/resource/Orson_Welles http://xmlns.com/foaf/0.1/name
http://dbpedia.org/resource/Citizen_Kane http://dbpedia.org/ontology/director http://dbpedia.org/resource/Orson_Welles http://dbpedia.org/ontology/birthPlace
http://dbpedia.org/resource/Citizen_Kane http://dbpedia.org/ontology/director http://dbpedia.org/resource/Orson_Welles http://dbpedia.org/ontology/deathDate
http://dbpedia.org/resource/Citizen_Kane http://dbpedia.org/ontology/director http://dbpedia.org/resource/Orson_Welles http://dbpedia.org/ontology/deathPlace
http://dbpedia.org/resource/Citizen_Kane http://dbpedia.org/ontology/director http://dbpedia.org/resource/Orson_Welles http://dbpedia.org/ontology/restingPlace
http://dbpedia.org/resource/Citizen_Kane http://db

In [30]:
transfwduop = transfwd.groupby(["s","r_x", "r_y"]).agg({"o": pd.Series.nunique})
transfwduo = transfwd[["s","r_x","o"]].groupby(["s","r_x"]).agg({"o": pd.Series.nunique})

In [31]:
transfwduo = transfwduo.reset_index()
transfwduop = transfwduop.reset_index()

In [32]:
transfwdu = transfwduop.merge(transfwduo, left_on=["s","r_x"], right_on=["s","r_x"])
transfwduf = transfwdu.copy()
transfwduf["o"] = transfwdu["o_x"] / transfwdu["o_y"]
transfwduf.drop(["o_x", "o_y"], axis=1, inplace=True)

In [34]:
print transfwduf.r_x.unique()

tslice = transfwduf[:20]
for i, row in tslice.iterrows():
    pass
    print revxdic[row.s], revzdic[row.r_x], revydic[row.r_y], row.o

[  4  27  14  26  31  56  75  90  45 409]
http://dbpedia.org/resource/Larry_Peerce http://dbpedia.org/ontology/birthPlace http://www.w3.org/1999/02/22-rdf-syntax-ns#type 1.0
http://dbpedia.org/resource/Larry_Peerce http://dbpedia.org/ontology/birthPlace http://xmlns.com/foaf/0.1/name 1.0
http://dbpedia.org/resource/Larry_Peerce http://dbpedia.org/ontology/birthPlace http://dbpedia.org/property/name 0.666666666667
http://dbpedia.org/resource/Larry_Peerce http://dbpedia.org/ontology/birthPlace http://dbpedia.org/property/caption 0.666666666667
http://dbpedia.org/resource/Larry_Peerce http://dbpedia.org/ontology/birthPlace http://dbpedia.org/property/title 0.333333333333
http://dbpedia.org/resource/Larry_Peerce http://dbpedia.org/ontology/birthPlace http://dbpedia.org/ontology/country 0.666666666667
http://dbpedia.org/resource/Larry_Peerce http://dbpedia.org/ontology/birthPlace http://dbpedia.org/ontology/isPartOf 0.666666666667
http://dbpedia.org/resource/Larry_Peerce http://dbpedia.org/

In [35]:
transfwduf

,s,r_x,r_y,o
0,1,4,1,1.000000
1,1,4,2,1.000000
2,1,4,3,0.666667
3,1,4,17,0.666667
4,1,4,36,0.333333
5,1,4,37,0.666667
6,1,4,43,0.666667
7,1,4,44,0.666667
8,1,4,46,0.666667
9,1,4,47,0.333333


In [36]:
zinvoffset = - zdic["http://dbpedia.org/ontology/spouse"] + zdic["-http://dbpedia.org/ontology/spouse"]

In [37]:
transrev = triples.merge(allrelsandtypes, left_on="s", right_on="e")
transrev.drop(["e"], axis=1, inplace=True)
transrev["r_x"] += zinvoffset

In [70]:
transrev

,s,r_x,o,r_y
0,417,532,9145,1
1,417,532,9145,2
2,417,532,9145,31
3,417,532,9145,26
4,417,532,9145,33
5,417,532,9145,14
6,417,532,9145,56
7,417,532,9145,93
8,417,532,9145,75
9,417,532,9145,90


In [38]:
print transrev.r_x.unique()

tslice = transrev[:20]
for i, row in tslice.iterrows():
    pass
    print revxdic[row.o], revzdic[row.r_x], revxdic[row.s], revydic[row.r_y]

[532 527 515 557 591 576 505 528 910 546]
http://dbpedia.org/resource/Orson_Welles -http://dbpedia.org/ontology/director http://dbpedia.org/resource/Citizen_Kane http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://dbpedia.org/resource/Orson_Welles -http://dbpedia.org/ontology/director http://dbpedia.org/resource/Citizen_Kane http://xmlns.com/foaf/0.1/name
http://dbpedia.org/resource/Orson_Welles -http://dbpedia.org/ontology/director http://dbpedia.org/resource/Citizen_Kane http://dbpedia.org/ontology/director
http://dbpedia.org/resource/Orson_Welles -http://dbpedia.org/ontology/director http://dbpedia.org/resource/Citizen_Kane http://dbpedia.org/ontology/producer
http://dbpedia.org/resource/Orson_Welles -http://dbpedia.org/ontology/director http://dbpedia.org/resource/Citizen_Kane http://dbpedia.org/ontology/writer
http://dbpedia.org/resource/Orson_Welles -http://dbpedia.org/ontology/director http://dbpedia.org/resource/Citizen_Kane http://dbpedia.org/ontology/starring
http://dbpedi

In [39]:
transrevuop = transrev.groupby(["o","r_x", "r_y"]).agg({"s": pd.Series.nunique})
transrevuo = transrev[["s","r_x","o"]].groupby(["o","r_x"]).agg({"s": pd.Series.nunique})
transrevuo = transrevuo.reset_index()
transrevuop = transrevuop.reset_index()

In [40]:
print transrevuop.r_x.unique()

tslice = transrevuop[:20]
for i, row in tslice.iterrows():
    pass
    print revxdic[row.o], revzdic[row.r_x], revydic[row.r_y], row.s

#transrevuo

[546 528 532 515 527 576 591 557 505 910]
http://dbpedia.org/resource/King_County,_Washington -http://dbpedia.org/ontology/location http://www.w3.org/1999/02/22-rdf-syntax-ns#type 1
http://dbpedia.org/resource/King_County,_Washington -http://dbpedia.org/ontology/location http://xmlns.com/foaf/0.1/name 1
http://dbpedia.org/resource/King_County,_Washington -http://dbpedia.org/ontology/location http://dbpedia.org/property/name 1
http://dbpedia.org/resource/King_County,_Washington -http://dbpedia.org/ontology/location http://dbpedia.org/property/location 1
http://dbpedia.org/resource/King_County,_Washington -http://dbpedia.org/ontology/location http://dbpedia.org/ontology/location 1
http://dbpedia.org/resource/King_County,_Washington -http://dbpedia.org/ontology/location http://dbpedia.org/ontology/nearestCity 1
http://dbpedia.org/resource/King_County,_Washington -http://dbpedia.org/ontology/location http://dbpedia.org/property/type 1
http://dbpedia.org/resource/King_County,_Washington -ht

In [48]:
transrevu = transrevuop.merge(transrevuo, left_on=["o","r_x"], right_on=["o","r_x"])
transrevuf = transrevu.copy()
transrevuf["s"] = transrevu["s_x"] / transrevu["s_y"]
transrevuf.drop(["s_x", "s_y"], axis=1, inplace=True)
transrevuf = transrevuf[["r_y", "r_x", "o", "s"]]

In [49]:
print transrevuf.r_x.unique()

tslice = transrevuf[:20]
for i, row in tslice.iterrows():
    pass
    print revxdic[row.o], revzdic[row.r_x], revydic[row.r_y], row.s

[546 528 532 515 527 576 591 557 505 910]
http://dbpedia.org/resource/King_County,_Washington -http://dbpedia.org/ontology/location http://www.w3.org/1999/02/22-rdf-syntax-ns#type 1.0
http://dbpedia.org/resource/King_County,_Washington -http://dbpedia.org/ontology/location http://xmlns.com/foaf/0.1/name 1.0
http://dbpedia.org/resource/King_County,_Washington -http://dbpedia.org/ontology/location http://dbpedia.org/property/name 1.0
http://dbpedia.org/resource/King_County,_Washington -http://dbpedia.org/ontology/location http://dbpedia.org/property/location 1.0
http://dbpedia.org/resource/King_County,_Washington -http://dbpedia.org/ontology/location http://dbpedia.org/ontology/location 1.0
http://dbpedia.org/resource/King_County,_Washington -http://dbpedia.org/ontology/location http://dbpedia.org/ontology/nearestCity 1.0
http://dbpedia.org/resource/King_County,_Washington -http://dbpedia.org/ontology/location http://dbpedia.org/property/type 1.0
http://dbpedia.org/resource/King_County,_

In [50]:
transrevuf = transrevuf[["o", "r_x", "r_y", "s"]]

In [51]:
transrevuf

,o,r_x,r_y,s
0,0,546,1,1
1,0,546,2,1
2,0,546,3,1
3,0,546,32,1
4,0,546,45,1
5,0,546,214,1
6,0,546,285,1
7,0,546,505,1
8,0,546,507,1
9,0,546,1042,1


In [52]:
print transfwduf.columns
print transrevuf.columns

Index([u's', u'r_x', u'r_y', u'o'], dtype='object')
Index([u'o', u'r_x', u'r_y', u's'], dtype='object')


In [53]:
# to save: 
# transrevuf, transfwduf, fxdic, zdic, ydic

## SAVE EVERYTHING

In [54]:
fulltensorp1 = transfwduf.copy()
fulltensorp1.columns = ["x","z","y","v"]
fulltensorp2 = transrevuf.copy()
fulltensorp2.columns = ["x","z","y","v"]
concats = [fulltensorp1, fulltensorp2]

In [55]:
if inclents:
    fulltensorp3 = triples.copy()
    ons = pd.DataFrame(np.ones((triples.shape[0],)))
    fulltensorp3 = pd.concat([fulltensorp3, ons], axis=1)
    fulltensorp3.columns = ["x", "z", "y", "v"]
    concats.append(fulltensorp3)

In [56]:
triples.shape

(34582, 3)

In [57]:
fulltensor = pd.concat(concats, axis=0)
fulltensor = fulltensor.reset_index(drop=True)

In [58]:
print len(fulltensor)
# drop some properties
def retainfun(x):
    if x in map(lambda y: ydic[y], [
        "http://www.w3.org/1999/02/22-rdf-syntax-ns#type",
        "http://www.w3.org/2000/01/rdf-schema#seeAlso",
        "http://dbpedia.org/property/caption",
        "http://dbpedia.org/ontology/type",
        "http://xmlns.com/foaf/0.1/homepage",
        "-http://www.w3.org/2000/01/rdf-schema#seeAlso",
        "-http://dbpedia.org/property/imageCaption",
        "http://dbpedia.org/property/imageCaption",
        "http://dbpedia.org/property/imageSize",
        "http://dbpedia.org/property/imagesize",
        "http://www.w3.org/2002/07/owl#differentFrom",
        "http://dbpedia.org/property/website",
    ]):
        return False
    return True

fil = fulltensor.y.map(retainfun)
fulltensor = fulltensor[fil]
print len(fulltensor)

1792053
1710480


In [59]:
for qsdf in ydic.keys():
    pass
    #print qsdf

In [68]:
#fulltensor[(fulltensor.y < idxrelstart) & (fulltensor.x < idxrelstart)]
fulltensor = fulltensor.sort_values(["z", "x", "y"])

In [69]:
# save fulltensor as .ssd
name = "fulltensorext"
if inclents:
    name += "+ents"
name += ".ssd"
with open(name, "w") as outf:
    for row in fulltensor.iterrows():
        row = row[1]
        outf.write("%d %d %d %f\n" % (row.z, row.x, row.y, row.v))

In [62]:
# save the company for the tensor
with open("fulltensorext.apx.pkl", "w") as outf:
    pickle.dump({"xdic": xdic, "zdic": zdic, "ydic": ydic}, outf)

## PLAY WITH DATA

In [81]:
print fulltensor.z.unique()

tslice = fulltensor[:50]
for i, row in tslice.iterrows():
    pass
    print revxdic[row.x], revzdic[row.z], revydic[row.y], row.v

[  4  27  14  26  31  56  75  90  45 409 546 528 532 515 527 576 591 557
 505 910]
http://dbpedia.org/resource/Larry_Peerce http://dbpedia.org/ontology/birthPlace http://xmlns.com/foaf/0.1/name 1.0
http://dbpedia.org/resource/Larry_Peerce http://dbpedia.org/ontology/birthPlace http://dbpedia.org/property/name 0.666666666667
http://dbpedia.org/resource/Larry_Peerce http://dbpedia.org/ontology/birthPlace http://dbpedia.org/property/title 0.333333333333
http://dbpedia.org/resource/Larry_Peerce http://dbpedia.org/ontology/birthPlace http://dbpedia.org/ontology/country 0.666666666667
http://dbpedia.org/resource/Larry_Peerce http://dbpedia.org/ontology/birthPlace http://dbpedia.org/ontology/isPartOf 0.666666666667
http://dbpedia.org/resource/Larry_Peerce http://dbpedia.org/ontology/birthPlace http://dbpedia.org/property/subdivisionName 0.666666666667
http://dbpedia.org/resource/Larry_Peerce http://dbpedia.org/ontology/birthPlace http://dbpedia.org/property/country 0.333333333333
http://dbped